In [1]:
import sys
import os
import shutil
import urllib.request
import gzip
import shutil
import xmltodict

import apitools


In [1]:
from annotation import uniprot
uniprot.update()

C:\DATA\Varjosalo lab Dropbox\Kari S\WIP_inprogress\2022 Analyysialusta\dashtesting\app\db_handlers\annotation\uniprot.py:455: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  return pd.DataFrame(columns=headers[1:], data=alltext, index=pd.Series(index, name='Entry'))
C:\DATA\Varjosalo lab Dropbox\Kari S\WIP_inprogress\2022 Analyysialusta\dashtesting\app\db_handlers\annotation\uniprot.py:455: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  return pd.DataFrame(columns=headers[1:], data=alltext, index=pd.Series(index, name='Entry'))
C:\DATA\Varjosalo lab Dropbox\Kari S\WIP_inprogress\2022 Analyysialusta\dashtesting\app\db_handlers\annotation\uniprot.py:455: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version

In [63]:
r = requests.get('https://www.uniprot.org/help/publications')§a

In [65]:
r.headers

{'Server': 'nginx/1.21.6', 'Vary': 'User-Agent', 'Content-Type': 'text/html', 'Content-Encoding': 'br', 'Strict-Transport-Security': 'max-age=31536000; includeSubDomains', 'Date': 'Wed, 21 Dec 2022 15:26:29 GMT', 'ETag': '"63a2d492-44a"', 'Connection': 'keep-alive', 'Last-Modified': 'Wed, 21 Dec 2022 09:40:34 GMT', 'Content-Length': '1098'}

In [52]:
import requests
organism = 9606
r = requests.get(f"https://rest.uniprot.org/uniprotkb/search?query=organism_id:{organism}&format=fasta")
r.headers['X-UniProt-Release']

In [60]:
for k, v in r.headers.items():
    print(k)

Server
Vary
Cache-Control
x-cache
Content-Type
Access-Control-Allow-Credentials
Content-Encoding
Access-Control-Expose-Headers
X-API-Deployment-Date
Strict-Transport-Security
Date
X-UniProt-Release
Link
X-Total-Results
Transfer-Encoding
Access-Control-Allow-Origin
Accept-Ranges
Connection
Access-Control-Allow-Methods
Access-Control-Allow-Headers
X-UniProt-Release-Date


In [62]:
r.headers['X-UniProt-Release']

'2022_05'

In [2]:
apitools.get_save_location('MIPS')

'Datafiles\\MIPS'

In [ ]:
update()

In [28]:
import pandas as pd

def parse(xmlfile):
    interactions = []
    heads = ['Source database','Reference DB','Reference ID','interaction detection name',
        'interaction detection reference DB','interaction detection reference ID',]
    for i in range(1,3):
        heads.extend([
            f'Protein {i} confidence unit',
            f'Protein {i} confidence value',
            f'Protein {i} is overexpressed',
            f'Protein {i} is tagged',
            f'Protein {i} fullName',
            f'Protein {i} shortName',
            f'Protein {i} organism',
            f'Protein {i} xref DB',
            f'Protein {i} xref ID',
        ])
    for entry in doc['entrySet']['entry']['interactionList']['interaction']:
            interactions.append([
                'MIPS',
                entry['experimentList']['experimentDescription']['bibref']['xref']['primaryRef']['@db']
            ])
            try:
                interactions[-1].append(entry['experimentList']['experimentDescription']['bibref']
                                            ['xref']['primaryRef']['@id'])
            except KeyError:
                interactions[-1].append('')
            interactions[-1].extend([
                entry['experimentList']['experimentDescription']['interactionDetection']['names']
                                        ['shortLabel'],
                entry['experimentList']['experimentDescription']['interactionDetection']['xref']
                                        ['primaryRef']['@db'],
                entry['experimentList']['experimentDescription']['interactionDetection']['xref']
                                        ['primaryRef']['@id']
            ])
            for pdic in entry['participantList']['proteinParticipant']:
                interactions[-1].extend([
                    pdic['confidence']['@unit'],
                    pdic['confidence']['@value'],
                    pdic['isOverexpressedProtein'],
                    pdic['isTaggedProtein'],
                    pdic['proteinInteractor']['names']['fullName'],
                    pdic['proteinInteractor']['names']['shortLabel'],
                    pdic['proteinInteractor']['organism']['@ncbiTaxId'],
                    pdic['proteinInteractor']['xref']['primaryRef']['@db']
                ])
                try:
                    interactions[-1].append(pdic['proteinInteractor']['xref']['primaryRef']['@id'])
                except KeyError:
                    interactions[-1].append(None)

def download_and_parse_mips(outfile):
    """Will download MIPS interaction database from https://mips.helmholtz-muenchen.de/proj/ppi/ \
        and parse it into a file
    """
    xmlfile = outfile.replace('gz','xml')
    url = "http://mips.helmholtz-muenchen.de/proj/ppi/data/mppi.gz"
    if not os.path.isfile(outfile):
        urllib.request.urlretrieve(url, outfile)

    with gzip.open(outfile, 'rb') as f_in:
        with open(xmlfile, 'wb') as f_out:
            shutil.copyfileobj(f_in, f_out)
    do_over = True
    while
    interactions = parse(xmlfile)
    df = pd.DataFrame(data=interactions,columns=heads)
    .to_csv(outfile.replace('.gz','_interactions.tsv'),sep='\t',index=False)
def update() -> None:
    today: str = apitools.get_timestamp()
    outdir: str = apitools.get_save_location('MIPS')
    if len(apitools.get_files_newer_than(outdir, today, 30, namefilter='mppi.gz'))<1:
        outfile: str = os.path.join(outdir, f'{today}_mppi.gz')
        download_and_parse_mips(outfile)
    
def get_interactions(filter = None) -> dict:
    ret_dict: dict = {}
    with open(apitools.get_newest_file(namefilter = '_interactions.tsv')) as fil:
        for line in fil:
            int1, int2 = line.strip().split('\t')
            if int1 not in ret_dict:
                ret_dict[int1] = []
            if int2 not in ret_dict:
                ret_dict[int2] = []
            ret_dict[int1].append(int2)
            ret_dict[int2].append(int1)


In [29]:

outfile = os.path.join('Datafiles','MIPS','2022-12-21_mppi.gz')
download_and_parse_mips(outfile)

Datafiles\MIPS\2022-12-21_mppi.xml


In [7]:
update()

In [30]:
df = pd.read_csv(os.path.join('Datafiles','MIPS','2022-12-21_mppi_interactions.tsv'),sep='\t')

In [31]:
df.head()

,Source database,Reference DB,Reference ID,interaction detection name,interaction detection reference DB,interaction detection reference ID,Protein 1 confidence unit,Protein 1 confidence value,Protein 1 is overexpressed,Protein 1 is tagged,...,Protein 1 xref ID,Protein 2 confidence unit,Protein 2 confidence value,Protein 2 is overexpressed,Protein 2 is tagged,Protein 2 fullName,Protein 2 shortName,Protein 2 organism,Protein 2 xref DB,Protein 2 xref ID
0,MIPS,PubMed,9867828,two hybrid,MI,18,boolean,1,False,False,...,P27005,boolean,1,False,False,S100A9;CAGB;MRP14; calgranulin B (migration in...,2,10090,SP,P31725
1,MIPS,PubMed,9867828,two hybrid,MI,18,boolean,1,False,False,...,P31725,boolean,1,False,False,S100A8;CAGA;MRP8; calgranulin A (migration inh...,1,10090,SP,P27005
2,MIPS,PubMed,9867828,two hybrid,MI,18,boolean,1,False,False,...,P05109,boolean,1,False,False,S100A9;CAGB;MRP14; calgranulin B (migration in...,4,9606,SP,P06702
3,MIPS,PubMed,9867828,two hybrid,MI,18,boolean,1,False,False,...,P06702,boolean,1,False,False,S100A8;CAGA;MRP8; calgranulin A (migration inh...,3,9606,SP,P05109
4,MIPS,PubMed,10611319,copurification,MI,25,boolean,1,False,False,...,U47050,boolean,1,False,False,"IP3R-3; inositol 1,4,5-trisphosphate receptor,...",6,9606,GB,D26351


In [35]:
df.columns

Index(['Source database', 'Reference DB', 'Reference ID',
       'interaction detection name', 'interaction detection reference DB',
       'interaction detection reference ID', 'Protein 1 confidence unit',
       'Protein 1 confidence value', 'Protein 1 is overexpressed',
       'Protein 1 is tagged', 'Protein 1 fullName', 'Protein 1 shortName',
       'Protein 1 organism', 'Protein 1 xref DB', 'Protein 1 xref ID',
       'Protein 2 confidence unit', 'Protein 2 confidence value',
       'Protein 2 is overexpressed', 'Protein 2 is tagged',
       'Protein 2 fullName', 'Protein 2 shortName', 'Protein 2 organism',
       'Protein 2 xref DB', 'Protein 2 xref ID'],
      dtype='object')

In [36]:
df[df['Protein 1 xref ID']=='-'][['Protein 1 fullName','Protein 1 fullName']]

,Protein 1 fullName,Protein 1 shortName
71,; surface glycoprotein CD100,57
81,; retinoblastoma-associated protein p107 (pock...,72
88,; retinoblastoma-associated protein RB (retino...,71
90,; retinoblastoma-associated protein p107 (pock...,72
111,; retinoblastoma-associated protein RB (retino...,71
112,; retinoblastoma-associated protein RB (retino...,71
115,; Msb1 protein,93
186,HSP90; Chaperone protein hsp90 (heat shock pro...,142
480,"ACT; actin, gamma",325
481,"ACT; actin, gamma",325


In [40]:
row[['Protein 1 xref ID']]

Protein 1 xref ID    P27005
Name: 0, dtype: object